# Ballpark Factor

Load the necessary packages

In [5]:
import requests
import lxml.html as lh
import pandas as pd

This is a function we'll use later to try turning data elements into integers, floats, or datetimes.

In [6]:
from datetime import datetime

def nice_data(d):
  #If we can turn it into an integer, then do that
  try:
    d=int(d)
  except:
  #If it's not an integer, it might be a float
    try:
      d = float(d)
    except:
      #If it's not a float, it might be a date
      try:
        d = datetime.strptime(d, '%m/%d/%Y')
      except:
        pass
  return(d)


the team names and numbers

In [7]:
teams =["BC", "GOC", "GRC", "HU", "INWU", "MAR", "MVNU", "SAU", "SFIN", "TAYL"]
t_nums = [1629, 1678, 1679, 1688, 1694, 1717, 1736, 1780, 1805, 1784]

Scrape the webpages

In [8]:
urls = ['http://www.dakstats.com/WebSync/Pages/Team/TeamSchedule.aspx?association=10&sg=MBA&sea=NAIMBA_2019&team=' +
        str(num) for num in t_nums]
#Create a handle, page, to handle the contents of the website
pages = [requests.get(url) for url in urls]
#Store the contents of the website under doc
docs = [lh.fromstring(page.content) for page in pages]
#Parse data that are stored between <table>..</table> of HTML
team_tables = [doc.xpath('//table') for doc in docs]

The for-loop below helps us find which tables on the website we need.  It appears that the header is the 33rd table, and the actual data is in the 35th table.

In [ ]:
for i in range(len(team_tables[2])):
  print(i, team_tables[2][i].xpath('.//tr/td//text()'))

In [25]:
headers = [tables[33].xpath('.//tr/td//text()') for tables in team_tables]
print(headers[2])

['Date', 'Opponent', 'H/A', 'Time', 'W/L']


In [26]:
team_elements = [tables[35].xpath('.//tr/td//text()') for tables in team_tables]
print(team_elements[2][:10])

['\r\n    ', '2/27/2019', ' Lourdes (Ohio)', 'N', ' 3-4 ', 'L', '\r\n          ', '\r\n    ', '\r\n    ', '\r\n    ']


In [27]:
team_good_els = [[nice_data(i) for i in elements if "\r" not in i] for elements in team_elements]
rows = [[good_els[i:i + 5] for i in range(0, len(good_els), 5)] for good_els in team_good_els]
rows[2][:2]

[[datetime.datetime(2019, 2, 27, 0, 0), ' Lourdes (Ohio)', 'N', ' 3-4 ', 'L'],
 [datetime.datetime(2019, 2, 27, 0, 0), ' Lourdes (Ohio)', 'N', ' 4-8 ', 'L']]

In [28]:
dfc = [pd.DataFrame(columns = headers[i], data = rows[i]) for i in range(len(headers))]
dfc[2][:5]

,Date,Opponent,H/A,Time,W/L
0,2019-02-27,Lourdes (Ohio),N,3-4,L
1,2019-02-27,Lourdes (Ohio),N,4-8,L
2,2019-03-02,Cornerstone (Mich.),N,3-4,L
3,2019-03-02,Trinity Baptist,N,5-1,W
4,2019-03-04,Michigan-Dearborn,N,13-1,W


Get only the conference games

In [54]:
conf_df = [df[df.Opponent.str.contains("*", regex = False)] for df in dfc]
conf_df[3][:5]

,Date,Opponent,H/A,Time,W/L
9,2019-03-08,Taylor (Ind.) *,A,7-6,W
10,2019-03-11,Taylor (Ind.) *,A,2-0,W
11,2019-03-11,Taylor (Ind.) *,A,1-5,L
12,2019-03-18,Spring Arbor (Mich.) *,H,6-5,W
13,2019-03-19,Spring Arbor (Mich.) *,H,8-0,W


This is a second web sraping method using Beautifulsoup4

In [9]:
from bs4 import BeautifulSoup

In [15]:
soup = BeautifulSoup(pages[2].content)

In [29]:
tables = [
    [
        [td.get_text(strip=True) for td in tr.find_all('td')] 
        for tr in table.find_all('tr')
    ] 
    for table in soup.find_all('table')
]

In [ ]:
for i in range(len(tables)):
  print(i, tables[i])

In [33]:
cols = tables[33][0]
cols

['Date', 'Opponent', 'H/A', 'Time', 'W/L']

In [38]:
rows = [r[:5] for r in tables[35][1::2]]
rows

[['2/27/2019', 'Lourdes (Ohio)', 'N', '3-4', 'L'],
 ['2/27/2019', 'Lourdes (Ohio)', 'N', '4-8', 'L'],
 ['3/2/2019', 'Cornerstone (Mich.)', 'N', '3-4', 'L'],
 ['3/2/2019', 'Trinity Baptist', 'N', '5-1', 'W'],
 ['3/4/2019', 'Michigan-Dearborn', 'N', '13-1', 'W'],
 ['3/5/2019', 'Rochester (Mich.)', 'N', '24-4', 'W'],
 ['3/6/2019', 'Robert Morris (Ill.)', 'N', '10-9', 'W'],
 ['3/8/2019', 'Bethel (Ind.) *', 'N', '13-6', 'W'],
 ['3/9/2019', 'Bethel (Ind.) *', 'N', '14-2', 'W'],
 ['3/9/2019', 'Bethel (Ind.) *', 'N', '3-1', 'W'],
 ['3/14/2019', 'Taylor (Ind.) *', 'A', '5-15', 'L'],
 ['3/16/2019', 'Taylor (Ind.) *', 'A', '2-10', 'L'],
 ['3/16/2019', 'Taylor (Ind.) *', 'A', '3-9', 'L'],
 ['3/23/2019', 'Spring Arbor (Mich.) *', 'A', '2-4', 'L'],
 ['3/25/2019', 'Spring Arbor (Mich.) *', 'A', '1-11', 'L'],
 ['3/25/2019', 'Spring Arbor (Mich.) *', 'A', '6-7', 'L'],
 ['3/26/2019', 'IU Kokomo (Ind.)', 'A', '0-3', 'L'],
 ['3/26/2019', 'IU Kokomo (Ind.)', 'A', '0-5', 'L'],
 ['3/28/2019', 'Huntington (In